### ANN/MLP 모델 구현
- 구성 : 전결합층(FC Layer)을 여러개로 구성
- 학습 : 회귀

[1] 모듈 로딩 및 데이터 준비

In [24]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [25]:
### 데이터 준비
DATA_FILE='./iris.csv'

### 데이터 로딩
### Header 파라미터 : 데이터 파일의 첫번째 줄이 컬럼명 여부 체크
### usecols 파라미터 : 로딩하고 싶은 컬럼 지정
iris_DF = pd.read_csv(DATA_FILE, header=None, usecols=[0,1,2,3])
iris_DF.head(5)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


[2] 모델 클래스 설계 및 정의
- 전결합층 클래스 => DenseModel
- 컬렉션 클래스 => SeqenceModel

In [26]:
### 전결합층 클래스 

class DenseModel(tf.Module):
    def __init__(self, input_nums, out_nums):
        super().__init__()
        self.W = tf.Variable(tf.random.normal((input_nums, out_nums)))
        self.b = tf.Variable(tf.zeros((out_nums)))
    
    def __call__(self, input):
        wbsum = tf.matmul(input, self.W) + self.b

        return wbsum

In [27]:
### -------------------------------------------------------
### 컬렉션 클래스
### 클래스 목적 : 모델의 층 구성 클래스
### 클래스 이름 : SequenceModel
### 클래스 속성 : in_layer, d_layer, out_layer
### 클래스 기능 : 층별 학습 진행 후 결과 전달 즉, 전방향 학습 기능
### -------------------------------------------------------

class SequenceModel(tf.Module):
    #인스턴스 생성 메서드
    def __init__(self, in_nums, out_nums):
        super().__init__()
        
        #입력층 : 피쳐개수, 퍼셉트론개수
        self.in_layer = DenseModel(in_nums, in_nums*2)

        #은닉층 : 피쳐개수, 퍼셉트론개수
        self.hd_layer = DenseModel(in_nums*2, in_nums)
        
        #출력층 : 피쳐개수, 타켓개수
        self.out_layer = DenseModel(in_nums,out_nums)

    # 순방향/전방향 학습 메서드
    def __call__(self, data):
        #입력층 학습
        wbsum = self.in_layer(data)
        output = tf.nn.relu(wbsum)

        #은닉층 학습
        wbsum = self.hd_layer(output)
        output = tf.nn.relu(wbsum)

        #출력층 학습
        wbsum = self.out_layer(output)

        return wbsum

[3] 학습 준비
- 데이터셋 -> 피쳐와 타켓 분리
- 인스턴스/객체 -> 모델 인스턴스, 최적화 인스턴스, 손실함수 인스턴스
- 학습 관련 함수들 -> 학습 함수, 검증함수, 예측함수

[3-1] 피쳐와 타켓 데이터셋 분리

In [33]:
### 텐서화 - 변경불가 데이터 즉, tf.constant()
# DataFrame -> Numpy ndarray
feature_NP = iris_DF[ iris_DF.columns[:-1] ].values
target_NP = iris_DF[ iris_DF.columns[-1:] ].values #1

print(f'feature_NP shape -> {feature_NP.shape}, target_NP shape -> {target_NP.shape}')

feature_NP shape -> (150, 3), target_NP shape -> (150, 1)


In [34]:
#Numpy ndarray --> Tensor
feature_TS = tf.constant(feature_NP, dtype=tf.float32)
target_TS = tf.constant(target_NP, dtype=tf.float32)

print(f'feature_TS shape -> {feature_NP.shape}, target_TS shape -> {target_NP.shape}')

feature_TS shape -> (150, 3), target_TS shape -> (150, 1)


[3-2] 모델 인스턴스 생성

In [35]:
# 입력 피쳐 수와 츨력 층 퍼셉트론 수 즉, 타겟 수
IN_NUMS, OUT_NUMS = feature_TS.shape[1], target_TS.shape[1]

print(f'IN_NUMS -> {IN_NUMS}, OUT_NUMS -> {OUT_NUMS}')

IN_NUMS -> 3, OUT_NUMS -> 1


In [36]:
# 모델 인스턴스 생성
iris_Model = SequenceModel(IN_NUMS, OUT_NUMS)

In [37]:
# 테스트
y_pre = iris_Model(feature_TS)

print(f'{y_pre}')

[[-0.8403568 ]
 [-1.4833374 ]
 [-0.8247688 ]
 [-0.97252965]
 [-0.534822  ]
 [-0.6454053 ]
 [-0.38758206]
 [-0.94061995]
 [-1.0305622 ]
 [-1.3516853 ]
 [-0.90871   ]
 [-0.7353482 ]
 [-1.3569522 ]
 [-0.58253336]
 [-0.7343075 ]
 [-0.03381538]
 [-0.45541382]
 [-0.8403568 ]
 [-1.2037115 ]
 [-0.35040522]
 [-1.5411563 ]
 [-0.52955484]
 [ 0.16070914]
 [-1.3411508 ]
 [-0.8778422 ]
 [-1.7047172 ]
 [-0.9881177 ]
 [-1.0142393 ]
 [-1.1458924 ]
 [-0.967262  ]
 [-1.2727976 ]
 [-1.4461601 ]
 [ 0.06066036]
 [-0.0918479 ]
 [-1.3516853 ]
 [-1.1564264 ]
 [-1.2983999 ]
 [-1.3516853 ]
 [-0.8039143 ]
 [-1.0670052 ]
 [-0.66647387]
 [-2.184349  ]
 [-0.44561434]
 [-0.80896735]
 [-0.5403967 ]
 [-1.3569522 ]
 [-0.39790297]
 [-0.74588203]
 [-0.78232574]
 [-1.0722718 ]
 [-4.87186   ]
 [-4.190056  ]
 [-4.8993673 ]
 [-4.102211  ]
 [-4.741869  ]
 [-3.857099  ]
 [-4.0033326 ]
 [-3.240377  ]
 [-4.7435284 ]
 [-3.33285   ]
 [-3.8067737 ]
 [-3.8166685 ]
 [-4.7531924 ]
 [-4.2142425 ]
 [-3.5099664 ]
 [-4.6089063 ]
 [-3.53409